In [1]:
import os
import torch
import torch.nn as nn
from torch.nn import functional as F
import time

#TODO Cross_entropy

In [2]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 1000
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1337)
device

'cuda'

In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Data exploration and Tokenizer

In [4]:
print("Length of dataset:", len(text))

Length of dataset: 1115394


In [5]:
print("First 1000 characters")
print(text[:1000])

First 1000 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in

In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# Create a mapping from characters to integeres (Tokenizer)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]          #ENCODER: string->integer
decode = lambda l: ''.join([itos[i] for i in l])  #DECODER: integer->string

print(encode("hii there"))
print(decode(encode("hii there")))


[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
# Train test split
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

# Model Building

In [10]:
#Generate a small batch of input x and target y
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [11]:
class Head(nn.Module):
    """ one head of self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # Get actual batch size, sequence length, and embedding dimension from input
        B, T, C = x.shape
        
        # Use self.key instead of undefined key variable
        k = self.key(x)   # (B, T, head_size)
        q = self.query(x) # (B, T, head_size)
        
        # compute attention scores
        wei = q @ k.transpose(-2, -1) # (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        
        # Perform weighted aggregation of the values
        v = self.value(x) # (B, T, head_size)
        out = wei @ v     # (B, T, head_size)
        
        return out

In [12]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [13]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [14]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [15]:
class miniGPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        # Get embeddings
        tok_embd = self.token_embedding_table(idx) # (B, T, C)
        pos_embd = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_embd + pos_embd # (B, T, C)
        
        # Apply self-attention
        x = self.blocks(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:] if idx.size(1) > block_size else idx
            
            # get predictions
            logits, _ = self(idx_cond)
            
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        
        return idx

In [16]:
model = miniGPT()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

10.788929 M parameters


In [17]:
# Create optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training

In [19]:
start_time = time.time()
context = torch.zeros((1, 1), dtype=torch.long, device=device)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"\nstep {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        print("------------")
        generated = model.generate(context, max_new_tokens=200)
        print(decode(generated[0].tolist()))
        print("------------")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.6f} seconds")


step 0: train loss 4.4948, val loss 4.4894
------------

SaVxux&f&&;gXYUCZo

DhDigFmHHpkwkBj!'zqSbUrDynIP.'qZU.nIOMm&LHKHP;iqkjnH nLRT!PBC!DAyCaDbu$hb&KIGEJr.Ca-!$SrSgzoQCZcn
yw!Ku'WP.PKxFxdFfRjI.D
iOmLBQzFmy
Ib, Oi
ErLiEk!cQjZrujxHmshIbMUTrTpvrnjwF:?ag.cpa
------------

step 1000: train loss 1.5439, val loss 1.7256
------------

Went when we There sahwe; ye recourself.

BENVOLINGBROKE:
I'll not eyes, the proff, since say.

QUEEN ELIZABET:
And I be caust Edward graou
't am behousat hone not you the?

GSAMPTER:
Lood sance now d
------------

step 2000: train loss 1.3411, val loss 1.5587
------------

What haFird never shands; where hath these
pamising strichly sook to cam to head; 'tis join
holy follower thus fair, a love in York of;
And then pardon use.

LUCIO:
As I trember's is was the lord rise
------------

step 3000: train loss 1.2402, val loss 1.5035
------------

Rest vow revense the general which thrival,
To see himbrace all: I'll see him out
For't: I have someture to Romeo.

R

# Generate

In [21]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)

# Generate text
generated = model.generate(context, max_new_tokens=2000)
print(decode(generated[0].tolist()))

# save
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist())) # -w to file


Diddling with surmised, what born butcher pays,
On he after, embracement. I fear to kill my tongue,
Farewell will not be so means of money.
My lord, have fear'd what I dreathed that broken in his
Deconcile and rease before the common's fault: their
I am never scarled, would I be the merry too;
And yea, to have learn'd your so melt folt.

DUKE OF AUMERLE:
Thine own gates can,
Are no little satisfied her;
Ard-every mind of rebells, and he's eneminate
With Lady Bona spark: therefore wixt the best
I restore the beholder.
Madam, Prisa your Trium.

LADY ANNE:
Still, fom you thit make not: I am indeed,
that your loss master, for I keep my shorout daughter.

CAPULET:
What news' Why, are, upon action!
Even never her voice, besides, and to cryining
By tries!

Post:
What's marrier, your lord.

WARWICK:
What may shall does verit
That go your honour, could hold him so mucal spers,
Nay, here's a money; therefore in puty title;
But for thy wolve I prove to shrift herward,
To-morrows; he'll follow yo

# Demonstration of token input

In [31]:
B,T,C = 4,8,2 #batch,time,channel
x = torch.randn(B,T,C)

In [ ]:
# Version 1, slow
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [ ]:
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b # matrix multiplication
print('a=')
print(a)
print('---')
print('b=')
print(b)
print('---')
print('c=')
print(c)

In [ ]:
# Version 2
torch.manual_seed(1337)
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (Bash, T, T) @ (B, T, C) ---> (B, T, C)
torch.allclose(xbow[0], xbow2[0])

In [ ]:
# Version 3: Use softmax
torch.manual_seed(1337)
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow[0], xbow3[0])

In [30]:
# Version 4: self-attention (!!)
torch.manual_seed(1337)
B,T,C = 4, 8, 32
x = torch.randn(B,T,C)

# Head of self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1)   # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)

v = value(x)
out = wei @ v
#out = wei @ x


tensor([[[-1.5713e-01,  8.8009e-01,  1.6152e-01, -7.8239e-01, -1.4289e-01,
           7.4676e-01,  1.0068e-01, -5.2395e-01, -8.8726e-01,  1.9068e-01,
           1.7616e-01, -5.9426e-01, -4.8124e-01, -4.8598e-01,  2.8623e-01,
           5.7099e-01],
         [ 8.3212e-01, -8.1437e-01, -3.2425e-01,  5.1913e-01, -1.2520e-01,
          -4.8982e-01, -5.2867e-01, -3.1393e-02,  1.0723e-01,  8.2692e-01,
           8.1322e-01, -2.7132e-02,  4.7754e-01,  4.9801e-01, -1.3769e-01,
           1.4025e+00],
         [ 6.0346e-01, -2.4995e-01, -6.1588e-01,  4.0678e-01,  3.3283e-01,
          -3.9097e-01,  1.3119e-01,  2.1718e-01, -1.2991e-01, -8.8281e-01,
           1.7242e-01,  4.6522e-01, -4.2710e-01, -7.6754e-02, -2.8524e-01,
           1.3875e+00],
         [ 6.6568e-01, -7.0960e-01, -6.0986e-01,  4.3484e-01,  8.9754e-01,
          -9.2983e-01,  6.8324e-02,  1.8632e-01,  5.4002e-01,  2.4271e-01,
          -6.9225e-01,  4.9774e-01,  4.8503e-01,  6.6076e-01,  8.7671e-01,
           7.4567e-02],
    